In [1]:
import pandas as pd
import numpy as np

In [2]:
# # This mounts your Google Drive to the Colab VM.
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# 파일 경로와 변수명을 딕셔너리로 정의
file_var_map = {
    './data/시황전망_news_크롤링_전처리_v1.xlsx': '시황전망',
    './data/기업종목분석_news_크롤링_전처리_v1.xlsx': '기업종목분석',
    './data/해외증시_news_크롤링_전처리_v1.xlsx': '해외증시',
    './data/채권선물_news_크롤링_전처리_v1.xlsx': '채권선물',
    './data/환율_news_크롤링_전처리_v1.xlsx': '환율'
}

# 파일 읽어서 딕셔너리에 저장
dataframes = {}
for file_path, var_name in file_var_map.items():
    df = pd.read_excel(file_path)
    df['구분'] = var_name
    dataframes[var_name] = df

# 모든 DataFrame 하나로 합치기
news_df = pd.concat(dataframes.values())
news_df.head(3)

,제목,뉴스 출처,날짜,뉴스 본문,관련 href,preprocessed_content,preprocessed_title,구분
0,"\n“주워담기 바쁘다”...외국인 뭉칫돈 몰리는 이 종목, 이번엔 오를까\n",매일경제,2023-08-07 22:01,잇단 목표가 상향 네이버 5% 상승외국인 투자자 186억원어치 순매수에코프로 등 2...,https://finance.naver.com/news/news_read.naver...,잇단 목표가 상향 네이버 5% 상승외국인 투자자 186억원어치 순매수에코프로 등 2...,"주워담기 바쁘다 외국인 뭉칫돈 몰리는 이 종목, 이번엔 오를까",시황전망
1,"\n""北, 동맹국 러 방산업체까지 해킹""<로이터>\n",서울경제,2023-08-07 21:21,[서울경제] 북한 해커집단이 러시아 주요 미사일 개발업체의 방화벽을 비밀리에 뚫는 ...,https://finance.naver.com/news/news_read.naver...,북한 해커집단이 러시아 주요 미사일 개발업체의 방화벽을 비밀리에 뚫는 데 성공한 것...,"北, 동맹국 러 방산업체까지 해킹 <로이터>",시황전망
2,"\n'훈센 장남' 훈 마넷, 총리로 공식 지명\n",서울경제,2023-08-07 20:56,[서울경제] 캄보디아 국왕이 훈센 총리의 장남인 훈 마넷을 차기 총리로 공식 지명했...,https://finance.naver.com/news/news_read.naver...,캄보디아 국왕이 훈센 총리의 장남인 훈 마넷을 차기 총리로 공식 지명했다.7일 AF...,"훈센 장남 훈 마넷, 총리로 공식 지명",시황전망


In [6]:
news_df['날짜'] = pd.to_datetime(news_df['날짜'])
news_df['시간'] = news_df['날짜'].dt.time  # 날짜 컬럼을 활용하여 시간 부분 추출
news_df['년월'] = news_df['날짜'].dt.date  # 날짜 컬럼을 활용하여 날짜 부분 추출
news_df.tail(3)

,제목,뉴스 출처,날짜,뉴스 본문,관련 href,preprocessed_content,preprocessed_title,구분,시간,년월
37,\n7월 외환보유액 2개월째 증가…3.5억달러 늘어난 4천218억달러\n,연합뉴스,2023-08-03 14:32:00,"(서울=연합뉴스) 박동주 기자 = 최근 미국 달러화 약세, 외화자산 운용수익 증가 ...",https://finance.naver.com/news/news_read.naver...,"최근 미국 달러화 약세, 외화자산 운용수익 증가 등으로 우리나라 외환보유액이 2개월...",7월 외환보유액 2개월째 증가 3.5억달러 늘어난 4천218억달러,환율,14:32:00,2023-08-03
38,\n7월 외환보유액…3.5억달러 늘어난 4천218억달러\n,연합뉴스,2023-08-03 14:32:00,"(서울=연합뉴스) 박동주 기자 = 최근 미국 달러화 약세, 외화자산 운용수익 증가 ...",https://finance.naver.com/news/news_read.naver...,"최근 미국 달러화 약세, 외화자산 운용수익 증가 등으로 우리나라 외환보유액이 2개월...",7월 외환보유액 3.5억달러 늘어난 4천218억달러,환율,14:32:00,2023-08-03
39,\n7월 외환보유액…3.5억달러 늘어난 4천218억달러\n,연합뉴스,2023-08-03 14:32:00,"(서울=연합뉴스) 박동주 기자 = 최근 미국 달러화 약세, 외화자산 운용수익 증가 ...",https://finance.naver.com/news/news_read.naver...,"최근 미국 달러화 약세, 외화자산 운용수익 증가 등으로 우리나라 외환보유액이 2개월...",7월 외환보유액 3.5억달러 늘어난 4천218억달러,환율,14:32:00,2023-08-03


In [7]:
def Date_Time_select(news_df, start_date, end_date):
    from datetime import datetime, time

    # '시작 날짜'와 '끝 날짜'를 datetime 형식으로 변환
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d %H:%M')
    end_datetime = datetime.strptime(end_date, '%Y-%m-%d %H:%M')

    # '시간' 컬럼을 datetime 형식으로 변환 후 필터링
    news_df['날짜'] = pd.to_datetime(news_df['날짜'])  # 시간 열을 datetime 형식으로 변환
    filtered_news_df = news_df[(start_datetime <= news_df['날짜']) & (news_df['날짜'] <= end_datetime)]

    return filtered_news_df

In [13]:
start_date = '2023-08-07 09:00'
end_date = '2023-08-08 17:00'
news_df = Date_Time_select(news_df, start_date, end_date)
news_df = news_df.sort_values(by='날짜')
print(news_df['날짜'].unique().min(), '~', news_df['날짜'].unique().max())

2023-08-07T09:00:00.000000000 ~ 2023-08-08T17:00:00.000000000


In [14]:
# 부친상 / 이벤트 들어간 기사는 행 삭제
# NH證, 횡성군 마을공동체에 냉장고 36대 기부
# 상장기념식
# 취약계층 쿨매트 기부

# 삭제할 키워드 리스트
keywords_to_delete = ['이벤트', '상장기념식', '부친상']

# 특정 키워드가 제목 또는 뉴스 본문에 포함된 행 삭제
for keyword in keywords_to_delete:
    if keyword != '부친상':
        news_df = news_df[~ (news_df['preprocessed_title'].str.contains(keyword, na=False, case=False))]
    else:
        news_df = news_df[~ (news_df['preprocessed_content'].str.contains(keyword, na=False, case=False))]

print(news_df.shape)

(648, 10)


In [15]:
# 뉴스 본문/제목에 1글자 이상
news_df = news_df[news_df['preprocessed_title'].str.len() > 1]
news_df = news_df[news_df['preprocessed_content'].str.len() > 1]

# 동일한 뉴스 제목은 삭제
news_df = news_df.drop_duplicates(subset=['preprocessed_title'])

news_df.shape

(589, 10)

In [16]:
news_df = news_df.reset_index(drop=True)

In [17]:
print(news_df['preprocessed_content'][1])
print(news_df['뉴스 본문'][1])

현대차증권은 7일 동원F&B에 대해 참치 실적이 견조한데 밸류에이션이 과도한 저평가상태라고 평가했다. 이에 따라 투자의견 매수 와 목표주가 4만5000원을 유지했다. 동원F&B의 2분기 연결 매출액은 1조520억원, 영업이익은 274억원으로 전년 동기 대비 각각 9.1%, 22.4% 증가한 것으로 집계됐다. 이는 시장 컨센서스를 상회한다. 하희지 연구원은 주력 제품인 참치 매출이 약 15% 성장해 일반 식품 외형 성장을 견인했다 며 높은 수준의 어가 및 환율에 따른 원가 부담에도 참치 가격 인상 효과 및 판매량 증가로 영업이익도 15% 성장한 것으로 추정된다 고 말했다. 이어 그는 하반기에도 견조한 참치 및 내식 식재료 카테고리 성장 흐름을 기대할 수 있다 면서도 다소 높은 수준의 어가 및 최근 상승한 원유가로 인한 원가 부담은 당분간 지속될 것 이라고 판단했다. 아울러 현재 주가는 12개월 포워드 PER 이 5배 수준으로 과도하게 저평가된 상태라고 분석했다. 하 연구원은 원가 부담 상쇄하는 이익 레버리지가 가능하다면 밸류에이션 정상화가 가능할 것 이라고 내다봤다.
현대차증권은 7일 동원F&B에 대해 참치 실적이 견조한데 밸류에이션이 과도한 저평가상태라고 평가했다. 이에 따라 투자의견 '매수'와 목표주가 4만5000원을 유지했다. 동원F&B의 2분기 연결 매출액은 1조520억원, 영업이익은 274억원으로 전년 동기 대비 각각 9.1%, 22.4% 증가한 것으로 집계됐다. 이는 시장 컨센서스를 상회한다. 하희지 연구원은 "주력 제품인 참치 매출이 약 15% 성장해 일반 식품 외형 성장을 견인했다"며 "높은 수준의 어가 및 환율에 따른 원가 부담에도 참치 가격 인상 효과 및 판매량 증가로 영업이익도 15% 성장한 것으로 추정된다"고 말했다. 이어 그는 "하반기에도 견조한 참치 및 내식 식재료 카테고리 성장 흐름을 기대할 수 있다"면서도 "다소 높은 수준의 어가 및 최근 상승한 원유가로 인한 원가 부담은 당분간 지속될 것"이라고 판단했다. 아울러 현재 주가는 12개

In [ ]:
# news_df.to_excel('/content/drive/MyDrive/Colab Notebooks/병합_230725_뉴스.xlsx', index=False, engine='openpyxl')